## Download and Clean Data

Download Link: https://www.chicago.gov/city/en/dataset/crime.html

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in c:\users\malli\appdata\local\programs\python\python313\lib\site-packages (2.2.3)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
import numpy as np

data = pd.read_csv(r"C:\Users\malli\Documents\Education\M.S. Spatial Data Science\Semester 5\DSCI 551\Project\Crimes_-_One_year_prior_to_present.csv")
data.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,JH117298,01/16/2024 01:00:00 AM,038XX W DIVERSEY AVE,0810,THEFT,OVER $500,STREET,N,N,2524,35,06,1150337.0,1918345.0,41.931844,-87.722951,"(41.931843966, -87.722950868)"
1,JG561057,12/31/2023 04:30:00 PM,004XX N WABASH AVE,0460,BATTERY,SIMPLE,STREET,N,N,1834,42,08B,1176592.0,1902931.0,41.888994,-87.626935,"(41.888993854, -87.626934833)"
2,JG512939,11/21/2023 02:28:00 PM,056XX S ELIZABETH ST,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,RESIDENCE - YARD (FRONT / BACK),N,N,713,16,15,1168951.0,1867382.0,41.791613,-87.656025,"(41.791613294, -87.656024853)"
3,JG496628,11/08/2023 03:27:00 PM,059XX N GLENWOOD AVE,0460,BATTERY,SIMPLE,SCHOOL - PUBLIC BUILDING,Y,N,2013,48,08B,1165910.0,1939379.0,41.989244,-87.665120,"(41.989243623, -87.665119726)"
4,JG512358,11/21/2023 02:12:00 AM,049XX W SCHUBERT AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,ALLEY,N,N,2521,31,14,1143030.0,1917505.0,41.929679,-87.749824,"(41.929678531, -87.749824286)"


In [11]:
print("Original Column Names:")
print(df.columns.tolist())


Original Column Names:
['CASE#', 'DATE  OF OCCURRENCE', 'BLOCK', ' IUCR', ' PRIMARY DESCRIPTION', ' SECONDARY DESCRIPTION', ' LOCATION DESCRIPTION', 'ARREST', 'DOMESTIC', 'BEAT', 'WARD', 'FBI CD', 'X COORDINATE', 'Y COORDINATE', 'LATITUDE', 'LONGITUDE', 'LOCATION']


In [12]:
import re

def clean_column_names(columns):
    cleaned_columns = []
    for col in columns:
        # Strip leading and trailing spaces
        col = col.strip()
        # Replace multiple spaces with a single space
        col = re.sub(r'\s+', ' ', col)
        # Replace spaces with underscores
        col = col.replace(' ', '_')
        # Convert to lowercase
        col = col.lower()
        # Remove any remaining special characters (optional)
        col = re.sub(r'[^\w_]', '', col)
        cleaned_columns.append(col)
    return cleaned_columns


In [14]:
# Apply the cleaning function to the current columns
data.columns = clean_column_names(data.columns)

print("Cleaned Column Names:")
print(data.columns.tolist())


Cleaned Column Names:
['case', 'date_of_occurrence', 'block', 'iucr', 'primary_description', 'secondary_description', 'location_description', 'arrest', 'domestic', 'beat', 'ward', 'fbi_cd', 'x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'location']


In [15]:
print(data.columns.tolist())


['case', 'date_of_occurrence', 'block', 'iucr', 'primary_description', 'secondary_description', 'location_description', 'arrest', 'domestic', 'beat', 'ward', 'fbi_cd', 'x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'location']


## Dropping Unnecessary Columns

In [21]:
# Define columns to drop
columns_to_drop = [
    'case',
    'iucr',
    'block',
    'fbi_cd',
    'location_description',  # High number of missing values
    'location'               # Potentially redundant with latitude and longitude
]

# Drop the columns
df_cleaned = data.drop(columns=columns_to_drop)

print("Columns after Dropping:")
print(df_cleaned.columns.tolist())


Columns after Dropping:
['date_of_occurrence', 'primary_description', 'secondary_description', 'arrest', 'domestic', 'beat', 'ward', 'x_coordinate', 'y_coordinate', 'latitude', 'longitude']


## Handling Missing Values

In [22]:
# Check for missing values in the cleaned DataFrame
missing_values = df_cleaned.isnull().sum()
print("Missing Values per Column:")
print(missing_values)


Missing Values per Column:
date_of_occurrence         0
primary_description        0
secondary_description      0
arrest                     0
domestic                   0
beat                       0
ward                       0
x_coordinate             106
y_coordinate             106
latitude                 106
longitude                106
dtype: int64


In [23]:
# Drop rows where latitude or longitude is missing
df_cleaned = df_cleaned.dropna(subset=['latitude', 'longitude'])

# Optionally, reset the index after dropping rows
df_cleaned.reset_index(drop=True, inplace=True)

# Verify that there are no more missing values in latitude and longitude
remaining_missing = df_cleaned[['latitude', 'longitude']].isnull().sum()
print("Missing Values after Dropping Rows:")
print(remaining_missing)


Missing Values after Dropping Rows:
latitude     0
longitude    0
dtype: int64


In [24]:
# Convert 'date_of_occurrence' to datetime
df_cleaned['date_of_occurrence'] = pd.to_datetime(df_cleaned['date_of_occurrence'], errors='coerce')

# Verify the conversion
print(df_cleaned['date_of_occurrence'].dtype)


datetime64[ns]


## Review Summary Statistics

In [25]:
# Display summary statistics for numerical columns
print(df_cleaned.describe())

# Display summary statistics for categorical columns
print(df_cleaned.describe(include=['object', 'bool']))


                  date_of_occurrence           beat           ward  \
count                         257473  257473.000000  257473.000000   
mean   2024-05-11 12:31:53.459209472    1157.791749      23.181580   
min              2023-11-08 05:20:00     111.000000       1.000000   
25%              2024-02-11 23:55:00     533.000000      10.000000   
50%              2024-05-16 13:45:00    1034.000000      23.000000   
75%              2024-08-08 08:15:00    1732.000000      34.000000   
max              2024-11-06 00:00:00    2535.000000      50.000000   
std                              NaN     710.378220      13.943045   

       x_coordinate  y_coordinate       latitude      longitude  
count  2.574730e+05  2.574730e+05  257473.000000  257473.000000  
mean   1.165264e+06  1.887719e+06      41.847483     -87.669008  
min    1.092647e+06  1.813897e+06      41.644590     -87.934567  
25%    1.154006e+06  1.860396e+06      41.772386     -87.709872  
50%    1.167108e+06  1.894388e+06      

In [26]:
# Save the cleaned dataset to a new CSV file
df_cleaned.to_csv('chicago_crime_data_cleaned.csv', index=False)

# Optionally, display the first few rows of the cleaned dataset
print(df_cleaned.head())


   date_of_occurrence primary_description          secondary_description  \
0 2024-01-16 01:00:00               THEFT                      OVER $500   
1 2023-12-31 16:30:00             BATTERY                         SIMPLE   
2 2023-11-21 14:28:00   WEAPONS VIOLATION  UNLAWFUL POSSESSION - HANDGUN   
3 2023-11-08 15:27:00             BATTERY                         SIMPLE   
4 2023-11-21 02:12:00     CRIMINAL DAMAGE                     TO VEHICLE   

  arrest domestic  beat  ward  x_coordinate  y_coordinate   latitude  \
0      N        N  2524    35     1150337.0     1918345.0  41.931844   
1      N        N  1834    42     1176592.0     1902931.0  41.888994   
2      N        N   713    16     1168951.0     1867382.0  41.791613   
3      Y        N  2013    48     1165910.0     1939379.0  41.989244   
4      N        N  2521    31     1143030.0     1917505.0  41.929679   

   longitude  
0 -87.722951  
1 -87.626935  
2 -87.656025  
3 -87.665120  
4 -87.749824  
